# الفصل 7: بناء تطبيقات الدردشة
## بدء سريع مع Azure OpenAI API


## نظرة عامة
تم تكييف هذه الدفتر من مستودع [عينات Azure OpenAI](https://github.com/Azure/azure-openai-samples?WT.mc_id=academic-105485-koreyst) الذي يتضمن دفاتر تصل أيضًا إلى خدمة [OpenAI](notebook-openai.ipynb).

تعمل واجهة برمجة تطبيقات OpenAI بلغة بايثون مع Azure OpenAI أيضًا، مع بعض التعديلات. تعرّف على المزيد حول الاختلافات هنا: [كيفية التبديل بين نقاط نهاية OpenAI و Azure OpenAI باستخدام بايثون](https://learn.microsoft.com/azure/ai-services/openai/how-to/switching-endpoints?WT.mc_id=academic-109527-jasmineg)

لمزيد من أمثلة البدء السريع، يرجى الرجوع إلى الوثائق الرسمية لـ [بدء استخدام Azure OpenAI](https://learn.microsoft.com/azure/cognitive-services/openai/quickstart?pivots=programming-language-studio&WT.mc_id=academic-105485-koreyst)


## جدول المحتويات  

[نظرة عامة](../../../../07-building-chat-applications/python)  
[البدء باستخدام خدمة Azure OpenAI](../../../../07-building-chat-applications/python)  
[بناء أول موجه لك](../../../../07-building-chat-applications/python)  

[حالات الاستخدام](../../../../07-building-chat-applications/python)    
[1. تلخيص النص](../../../../07-building-chat-applications/python)  
[2. تصنيف النص](../../../../07-building-chat-applications/python)  
[3. توليد أسماء منتجات جديدة](../../../../07-building-chat-applications/python)  
[4. ضبط دقيق لمصنف](../../../../07-building-chat-applications/python)  
[5. التضمينات](../../../../07-building-chat-applications/python)

[المراجع](../../../../07-building-chat-applications/python)


### البدء مع خدمة Azure OpenAI

سيحتاج العملاء الجدد إلى [التقديم للحصول على الوصول](https://aka.ms/oai/access?WT.mc_id=academic-105485-koreyst) إلى خدمة Azure OpenAI.  
بعد اكتمال الموافقة، يمكن للعملاء تسجيل الدخول إلى بوابة Azure، وإنشاء مورد خدمة Azure OpenAI، والبدء في تجربة النماذج عبر الاستوديو  

[مصدر رائع للبدء بسرعة](https://techcommunity.microsoft.com/blog/educatordeveloperblog/azure-openai-service-is-now-generally-available/3719177?WT.mc_id=academic-105485-koreyst)


### أنشئ أول موجه لك  
ستوفر هذه التمرين القصير مقدمة أساسية لإرسال الموجهات إلى نموذج OpenAI لمهمة بسيطة "التلخيص".


**الخطوات**:  
1. قم بتثبيت مكتبة OpenAI في بيئة بايثون الخاصة بك  
2. قم بتحميل مكتبات المساعدة القياسية وقم بتعيين بيانات اعتماد الأمان النموذجية الخاصة بك لخدمة OpenAI التي أنشأتها  
3. اختر نموذجًا لمهمتك  
4. أنشئ موجهًا بسيطًا للنموذج  
5. أرسل طلبك إلى واجهة برمجة تطبيقات النموذج!


### ١. تثبيت OpenAI


> [!ملاحظة] هذه الخطوة غير ضرورية إذا قمت بتشغيل هذا الدفتر على Codespaces أو داخل Devcontainer


In [ ]:
%pip install openai python-dotenv

### ٢. استيراد مكتبات المساعدة وإنشاء بيانات الاعتماد


In [ ]:
import os
from openai import AzureOpenAI
import numpy as np
from dotenv import load_dotenv
load_dotenv()

#validate data inside .env file

client = AzureOpenAI(
  api_key=os.environ['AZURE_OPENAI_API_KEY'],  # this is also the default, it can be omitted
  api_version = "2023-05-15"
  )

### 3. العثور على النموذج المناسب  
يمكن لنماذج GPT-3.5-turbo أو GPT-4 فهم اللغة الطبيعية وتوليدها. تقدم الخدمة أربع قدرات للنماذج، كل منها بمستويات مختلفة من القوة والسرعة مناسبة لمهام مختلفة.

[نماذج Azure OpenAI](https://learn.microsoft.com/azure/cognitive-services/openai/concepts/models?WT.mc_id=academic-105485-koreyst)


In [ ]:
# Select the General Purpose curie model for text
model = os.environ['AZURE_OPENAI_DEPLOYMENT']

## 4. تصميم المطالبات  

"السحر في نماذج اللغة الكبيرة هو أنه من خلال تدريبها على تقليل خطأ التنبؤ هذا عبر كميات هائلة من النصوص، تنتهي النماذج بتعلم مفاهيم مفيدة لهذه التنبؤات. على سبيل المثال، تتعلم مفاهيم مثل"(1):

* كيفية التهجئة
* كيفية عمل القواعد النحوية
* كيفية إعادة الصياغة
* كيفية الإجابة على الأسئلة
* كيفية إجراء محادثة
* كيفية الكتابة بعدة لغات
* كيفية البرمجة
* إلخ.

#### كيفية التحكم في نموذج اللغة الكبير  
"من بين جميع المدخلات إلى نموذج اللغة الكبير، فإن أكثرها تأثيرًا هو مطالبة النص(1).

يمكن تحفيز نماذج اللغة الكبيرة لإنتاج مخرجات بعدة طرق:

- التعليمات: أخبر النموذج بما تريد
- الإكمال: حث النموذج على إكمال بداية ما تريد
- العرض: أرِ النموذج ما تريد، إما بـ:
- بعض الأمثلة في المطالبة
- مئات أو آلاف الأمثلة في مجموعة بيانات تدريبية للتعديل الدقيق



#### هناك ثلاث إرشادات أساسية لإنشاء المطالبات:

**أرِ وقل.** اجعل ما تريده واضحًا إما من خلال التعليمات أو الأمثلة أو مزيج من الاثنين. إذا كنت تريد من النموذج ترتيب قائمة من العناصر أبجديًا أو تصنيف فقرة حسب المشاعر، أرِه أن هذا ما تريده.

**قدّم بيانات ذات جودة.** إذا كنت تحاول بناء مصنف أو جعل النموذج يتبع نمطًا، تأكد من وجود عدد كافٍ من الأمثلة. تأكد من تدقيق أمثلتك — النموذج عادة ذكي بما يكفي لرؤية الأخطاء الإملائية الأساسية ويعطيك ردًا، لكنه قد يفترض أيضًا أن هذا مقصود وقد يؤثر على الرد.

**تحقق من إعداداتك.** تتحكم إعدادات درجة الحرارة وtop_p في مدى حتمية النموذج في توليد الرد. إذا كنت تطلب ردًا حيث يوجد جواب صحيح واحد فقط، فستريد ضبط هذه الإعدادات على قيم أقل. إذا كنت تبحث عن ردود أكثر تنوعًا، فقد ترغب في ضبطها على قيم أعلى. الخطأ الأول الذي يرتكبه الناس مع هذه الإعدادات هو افتراض أنها تحكم "الذكاء" أو "الإبداع".


المصدر: https://learn.microsoft.com/azure/ai-services/openai/overview


الصورة تنشئ أول موجه نصي لك!


### ٥. قدّم!


In [ ]:
# Create your first prompt
text_prompt = "Should oxford commas always be used?"

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":text_prompt},])

response.choices[0].message.content

### كرر نفس المكالمة، كيف تقارن النتائج؟


In [ ]:

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":text_prompt},])

response.choices[0].message.content

## تلخيص النص  
#### التحدي  
تلخيص النص بإضافة "tl;dr:" في نهاية مقطع النص. لاحظ كيف يفهم النموذج كيفية أداء عدد من المهام دون تعليمات إضافية. يمكنك تجربة مطالبات أكثر وصفًا من tl;dr لتعديل سلوك النموذج وتخصيص التلخيص الذي تتلقاه(3).  

أظهرت الأعمال الحديثة مكاسب كبيرة في العديد من مهام ومعايير معالجة اللغة الطبيعية من خلال التدريب المسبق على مجموعة كبيرة من النصوص يليه التخصيص الدقيق لمهمة محددة. على الرغم من أن الهيكلية عادة ما تكون غير معتمدة على المهمة، إلا أن هذه الطريقة لا تزال تتطلب مجموعات بيانات تخصيص دقيقة خاصة بالمهمة تحتوي على آلاف أو عشرات الآلاف من الأمثلة. بالمقابل، يمكن للبشر عمومًا أداء مهمة لغوية جديدة من خلال عدد قليل من الأمثلة أو تعليمات بسيطة - وهو أمر لا تزال أنظمة معالجة اللغة الطبيعية الحالية تكافح لتحقيقه إلى حد كبير. هنا نُظهر أن توسيع نماذج اللغة يحسن بشكل كبير الأداء غير المعتمد على المهمة مع عدد قليل من الأمثلة، وأحيانًا يصل إلى منافسة الأساليب السابقة المتقدمة في التخصيص الدقيق.  

tl;dr


# تمارين لعدة حالات استخدام  
1. تلخيص النص  
2. تصنيف النص  
3. توليد أسماء منتجات جديدة  
4. التضمينات  
5. ضبط دقيق لمصنف


In [ ]:
prompt = "Recent work has demonstrated substantial gains on many NLP tasks and benchmarks by pre-training on a large corpus of text followed by fine-tuning on a specific task. While typically task-agnostic in architecture, this method still requires task-specific fine-tuning datasets of thousands or tens of thousands of examples. By contrast, humans can generally perform a new language task from only a few examples or from simple instructions - something that current NLP systems still largely struggle to do. Here we show that scaling up language models greatly improves task-agnostic, few-shot performance, sometimes even reaching competitiveness with prior state-of-the-art fine-tuning approaches.\n\ntl;dr"


In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt},])

response.choices[0].message.content

## تصنيف النص  
#### التحدي  
تصنيف العناصر إلى فئات يتم توفيرها في وقت الاستدلال. في المثال التالي، نوفر كل من الفئات والنص المراد تصنيفه في الموجه (*playground_reference).

استفسار العميل: مرحبًا، أحد مفاتيح لوحة مفاتيح اللابتوب الخاصة بي انكسر مؤخرًا وسأحتاج إلى استبداله:

الفئة المصنفة:


In [ ]:
prompt = "Classify the following inquiry into one of the following: categories: [Pricing, Hardware Support, Software Support]\n\ninquiry: Hello, one of the keys on my laptop keyboard broke recently and I'll need a replacement:\n\nClassified category:"
print(prompt)

In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt},])

response.choices[0].message.content

## توليد أسماء منتجات جديدة
#### التحدي
إنشاء أسماء منتجات من كلمات أمثلة. هنا ندرج في الموجه معلومات عن المنتج الذي سنولد له أسماء. كما نوفر مثالًا مشابهًا لإظهار النمط الذي نرغب في استلامه. لقد قمنا أيضًا بضبط قيمة درجة الحرارة عالية لزيادة العشوائية والحصول على ردود أكثر ابتكارًا.

وصف المنتج: صانع مخفوق الحليب المنزلي  
كلمات البذور: سريع، صحي، مدمج.  
أسماء المنتجات: HomeShaker, Fit Shaker, QuickShake, Shake Maker

وصف المنتج: زوج من الأحذية يمكن أن يناسب أي مقاس قدم.  
كلمات البذور: قابل للتكيف، مناسب، أومني-فيت.


In [ ]:
prompt = "Product description: A home milkshake maker\nSeed words: fast, healthy, compact.\nProduct names: HomeShaker, Fit Shaker, QuickShake, Shake Maker\n\nProduct description: A pair of shoes that can fit any foot size.\nSeed words: adaptable, fit, omni-fit."

print(prompt)

In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt}])

response.choices[0].message.content

## التضمينات
سيعرض هذا القسم كيفية استرجاع التضمينات، وإيجاد التشابهات بين الكلمات، والجمل، والمستندات. لتشغيل دفاتر الملاحظات التالية، تحتاج إلى نشر نموذج يستخدم `text-embedding-ada-002` كنموذج أساسي وتعيين اسم نشره داخل ملف .env، باستخدام المتغير `AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT`.


### تصنيف النموذج - اختيار نموذج التضمين

**تصنيف النموذج**: {family} - {capability} - {input-type} - {identifier}  

{family}     --> text-embedding  (عائلة التضمينات)  
{capability} --> ada             (جميع نماذج التضمين الأخرى سيتم إيقافها في 2024)  
{input-type} --> n/a             (محدد فقط لنماذج البحث)  
{identifier} --> 002             (الإصدار 002)  

model = 'text-embedding-ada-002'


> [!ملاحظة] الخطوة التالية غير ضرورية إذا قمت بتشغيل هذا الدفتر على Codespaces أو داخل Devcontainer


In [ ]:
# Dependencies for embeddings_utils
%pip install matplotlib plotly scikit-learn pandas

In [ ]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

In [ ]:
text = 'the quick brown fox jumped over the lazy dog'
model= os.environ['AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT']
client.embeddings.create(input='[text]', model=model).data[0].embedding

In [ ]:

# compare several words
automobile_embedding  = client.embeddings.create(input='automobile', model=model).data[0].embedding
vehicle_embedding     = client.embeddings.create(input='vehicle', model=model).data[0].embedding
dinosaur_embedding    = client.embeddings.create(input='dinosaur', model=model).data[0].embedding
stick_embedding       = client.embeddings.create(input='stick', model=model).data[0].embedding

print(cosine_similarity(automobile_embedding, vehicle_embedding))
print(cosine_similarity(automobile_embedding, dinosaur_embedding))
print(cosine_similarity(automobile_embedding, stick_embedding))

## مقارنة مقال من مجموعة أخبار CNN اليومية
المصدر: https://huggingface.co/datasets/cnn_dailymail


In [ ]:
import pandas as pd
cnn_daily_articles = ['BREMEN, Germany -- Carlos Alberto, who scored in FC Porto\'s Champions League final victory against Monaco in 2004, has joined Bundesliga club Werder Bremen for a club record fee of 7.8 million euros ($10.7 million). Carlos Alberto enjoyed success at FC Porto under Jose Mourinho. "I\'m here to win titles with Werder," the 22-year-old said after his first training session with his new club. "I like Bremen and would only have wanted to come here." Carlos Alberto started his career with Fluminense, and helped them to lift the Campeonato Carioca in 2002. In January 2004 he moved on to FC Porto, who were coached by José Mourinho, and the club won the Portuguese title as well as the Champions League. Early in 2005, he moved to Corinthians, where he impressed as they won the Brasileirão,but in 2006 Corinthians had a poor season and Carlos Alberto found himself at odds with manager, Emerson Leão. Their poor relationship came to a climax at a Copa Sul-Americana game against Club Atlético Lanús, and Carlos Alberto declared that he would not play for Corinthians again while Leão remained as manager. Since January this year he has been on loan with his first club Fluminense. Bundesliga champions VfB Stuttgart said on Sunday that they would sign a loan agreement with Real Zaragoza on Monday for Ewerthon, the third top Brazilian player to join the German league in three days. A VfB spokesman said Ewerthon, who played in the Bundesliga for Borussia Dortmund from 2001 to 2005, was expected to join the club for their pre-season training in Austria on Monday. On Friday, Ailton returned to Germany where he was the league\'s top scorer in 2004, signing a one-year deal with Duisburg on a transfer from Red Star Belgrade. E-mail to a friend .',
                        '(CNN) -- Football superstar, celebrity, fashion icon, multimillion-dollar heartthrob. Now, David Beckham is headed for the Hollywood Hills as he takes his game to U.S. Major League Soccer. CNN looks at how Bekham fulfilled his dream of playing for Manchester United, and his time playing for England. The world\'s famous footballer has begun a five-year contract with the Los Angeles Galaxy team, and on Friday Beckham will meet the press and reveal his new shirt number. This week, we take an in depth look at the life and times of Beckham, as CNN\'s very own "Becks," Becky Anderson, sets out to examine what makes the man tick -- as footballer, fashion icon and global phenomenon. It\'s a long way from the streets of east London to the Hollywood Hills and Becky charts Beckham\'s incredible rise to football stardom, a journey that has seen his skills grace the greatest stages in world soccer. She goes in pursuit of the current hottest property on the sports/celebrity circuit in the U.S. and along the way explores exactly what\'s behind the man with the golden boot. CNN will look back at the life of Beckham, the wonderfully talented youngster who fulfilled his dream of playing for Manchester United, his marriage to pop star Victoria, and the trials and tribulations of playing for England. We\'ll look at the highs (scoring against Greece), the lows (being sent off during the World Cup), the Man. U departure for the Galacticos of Madrid -- and now the Home Depot stadium in L.A. We\'ll ask how Beckham and his family will adapt to life in Los Angeles -- the people, the places to see and be seen and the celebrity endorsement. Beckham is no stranger to exposure. He has teamed with Reggie Bush in an Adidas commercial, is the face of Motorola, is the face on a PlayStation game and doesn\'t need fashion tips as he has his own international clothing line. But what does the star couple need to do to become an accepted part of Tinseltown\'s glitterati? The road to major league football in the U.S.A. is a well-worn route for some of the world\'s greatest players. We talk to some of the former greats who came before him and examine what impact these overseas stars had on U.S. soccer and look at what is different now. We also get a rare glimpse inside the David Beckham academy in L.A, find out what drives the kids and who are their heroes. The perception that in the U.S.A. soccer is a "game for girls" after the teenage years is changing. More and more young kids are choosing the European game over the traditional U.S. sports. E-mail to a friend .',
                        'LOS ANGELES, California (CNN) -- Youssif, the 5-year-old burned Iraqi boy, rounded the corner at Universal Studios when suddenly the little boy hero met his favorite superhero. Youssif has always been a huge Spider-Man fan. Meeting him was "my favorite thing," he said. Spider-Man was right smack dab in front of him, riding a four-wheeler amid a convoy of other superheroes. The legendary climber of buildings and fighter of evil dismounted, walked over to Youssif and introduced himself. Spidey then gave the boy from a far-away land a gentle hug, embracing him in his iconic blue and red tights. He showed Youssif a few tricks, like how to shoot a web from his wrist. Only this time, no web was spun. "All right Youssif!" Spider-Man said after the boy mimicked his wrist movement. Other superheroes crowded around to get a closer look. Even the Green Goblin stopped his villainous ways to tell the boy hi. Youssif remained unfazed. He didn\'t take a liking to Spider-Man\'s nemesis. Spidey was just too cool. "It was my favorite thing," the boy said later. "I want to see him again." He then felt compelled to add: "I know it\'s not the real Spider-Man." This was the day of dreams when the boy\'s nightmares were, at least temporarily, forgotten. He met SpongeBob, Lassie and a 3-year-old orangutan named Archie. The hairy, brownish-red primate took to the boy, grabbing his hand and holding it. Even when Youssif pulled away, Archie would inch his hand back toward the boy\'s and then snatch it. See Youssif enjoy being a boy again » . The boy giggled inside a play area where sponge-like balls shot out of toy guns. It was a far different artillery than what he was used to seeing in central Baghdad, as recently as a week ago. He squealed with delight and raced around the room collecting as many balls as he could. He rode a tram through the back stages at Universal Studios. At one point, the car shook. Fire and smoke filled the air, debris cascaded down and a big rig skidded toward the vehicle. The boy and his family survived the pretend earthquake unscathed. "Even I was scared," the dad said. "Well, I wasn\'t," Youssif replied. The father and mother grinned from ear to ear throughout the day. Youssif pushed his 14-month-old sister, Ayaa, in a stroller. "Did you even need to ask us if we were interested in coming here?" Youssif\'s father said in amazement. "Other than my wedding day, this is the happiest day of my life," he said. Just a day earlier, the mother and father talked about their journey out of Iraq and to the United States. They also discussed that day nine months ago when masked men grabbed their son outside the family home, doused him in gas and set him on fire. His mother heard her boy screaming from inside. The father sought help for his boy across Baghdad, but no one listened. He remembers his son\'s two months of hospitalization. The doctors didn\'t use anesthetics. He could hear his boy\'s piercing screams from the other side of the hospital. Watch Youssif meet his doctor and play with his little sister » . The father knew that speaking to CNN would put his family\'s lives in jeopardy. The possibility of being killed was better than seeing his son suffer, he said. "Anything for Youssif," he said. "We had to do it." They described a life of utter chaos in Baghdad. Neighbors had recently given birth to a baby girl. Shortly afterward, the father was kidnapped and killed. Then, there was the time when some girls wore tanktops and jeans. They were snatched off the street by gunmen. The stories can be even more gruesome. The couple said they had heard reports that a young girl was kidnapped and beheaded --and her killers sewed a dog\'s head on the corpse and delivered it to her family\'s doorstep. "These are just some of the stories," said Youssif\'s mother, Zainab. Under Saddam Hussein, there was more security and stability, they said. There was running water and electricity most of the time. But still life was tough under the dictator, like the time when Zainab\'s uncle disappeared and was never heard from again after he read a "religious book," she said. Sitting in the parking lot of a Target in suburban Los Angeles, Youssif\'s father watched as husbands and wives, boyfriends and girlfriends, parents and their children, came and went. Some held hands. Others smiled and laughed. "Iraq finished," he said in what few English words he knows. He elaborated in Arabic: His homeland won\'t be enjoying such freedoms anytime soon. It\'s just not possible. Too much violence. Too many killings. His two children have only seen war. But this week, the family has seen a much different side of America -- an outpouring of generosity and a peaceful nation at home. "It\'s been a dream," the father said. He used to do a lot of volunteer work back in Baghdad. "Maybe that\'s why I\'m being helped now," the father said. At Universal Studios, he looked out across the valley below. The sun glistened off treetops and buildings. It was a picturesque sight fit for a Hollywood movie. "Good America, good America," he said in English. E-mail to a friend . CNN\'s Arwa Damon contributed to this report.'
]

cnn_daily_article_highlights = ['Werder Bremen pay a club record $10.7 million for Carlos Alberto .\nThe Brazilian midfielder won the Champions League with FC Porto in 2004 .\nSince January he has been on loan with his first club, Fluminense .',
                                'Beckham has agreed to a five-year contract with Los Angeles Galaxy .\nNew contract took effect July 1, 2007 .\nFormer English captain to meet press, unveil new shirt number Friday .\nCNN to look at Beckham as footballer, fashion icon and global phenomenon .',
                                'Boy on meeting Spider-Man: "It was my favorite thing"\nYoussif also met SpongeBob, Lassie and an orangutan at Universal Studios .\nDad: "Other than my wedding day, this is the happiest day of my life"'
]

cnn_df = pd.DataFrame({"articles":cnn_daily_articles, "highligths":cnn_daily_article_highlights})

cnn_df.head()

In [ ]:
article1_embedding    = client.embeddings.create(input=cnn_df.articles.iloc[0], model=model).data[0].embedding
article2_embedding    = client.embeddings.create(input=cnn_df.articles.iloc[1], model=model).data[0].embedding
article3_embedding    = client.embeddings.create(input=cnn_df.articles.iloc[2], model=model).data[0].embedding

print(cosine_similarity(article1_embedding, article2_embedding))
print(cosine_similarity(article1_embedding, article3_embedding))

# المراجع  
- [توثيق أزور - نماذج أزور OpenAI](https://learn.microsoft.com/azure/cognitive-services/openai/concepts/models?WT.mc_id=academic-105485-koreyst)  
- [أمثلة استوديو OpenAI](https://oai.azure.com/portal?WT.mc_id=academic-105485-koreyst)  


# للمزيد من المساعدة  
[فريق تسويق OpenAI](AzureOpenAITeam@microsoft.com)


# المساهمون
* لويس لي  


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**إخلاء المسؤولية**:  
تمت ترجمة هذا المستند باستخدام خدمة الترجمة الآلية [Co-op Translator](https://github.com/Azure/co-op-translator). بينما نسعى لتحقيق الدقة، يرجى العلم أن الترجمات الآلية قد تحتوي على أخطاء أو عدم دقة. يجب اعتبار المستند الأصلي بلغته الأصلية المصدر الموثوق به. للمعلومات الهامة، يُنصح بالاعتماد على الترجمة البشرية المهنية. نحن غير مسؤولين عن أي سوء فهم أو تفسير ناتج عن استخدام هذه الترجمة.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
